In [2]:
import os
import json
import glob
import h5py
import numpy as np
from tqdm.auto import tqdm

In [3]:
outdir = '../../data_prods/'
data_dir = '/home/ryan/Data'
subdir_list = [os.path.join(data_dir, o) for o in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir,o))]

print(subdir_list)

['/home/ryan/Data/run_2cDM_L3N256_DM_powerm2m2_sigma0.1', '/home/ryan/Data/run_2cDM_L3N512_HY_power00_sigma1', '/home/ryan/Data/run_2cDM_L3N256_HY_power00_sigma0.1', '/home/ryan/Data/run_SIDM_L3N256_DM_power0_sigma1', '/home/ryan/Data/lost+found', '/home/ryan/Data/run_2cDM_L3N256_DM_power00_sigma1_dir_3', '/home/ryan/Data/run_2cDM_L3N256_HY_powerm2m2_sigma10', '/home/ryan/Data/run_CDM_L3N256_DM_dir_7', '/home/ryan/Data/run_2cDM_L3N256_DM_power00_sigma1_dir_8', '/home/ryan/Data/run_CDM_L5N256_HY', '/home/ryan/Data/run_2cDM_L3N128_HY_power00_sigma0.1_10e-7', '/home/ryan/Data/run_2cDM_L3N256_DM_power00_sigma0.1', '/home/ryan/Data/run_2cDM_L3N128_HY_power00_sigma0.1_var4', '/home/ryan/Data/run_2cDM_L3N256_DM_power00_sigma1_dir_4', '/home/ryan/Data/run_CDM_710_HY', '/home/ryan/Data/run_2cDM_710_HY_powerm2m2_sigma1', '/home/ryan/Data/run_2cDM_L3N256_DM_power00_sigma1_dir_1', '/home/ryan/Data/run_CDM_L3N128_HY', '/home/ryan/Data/run_2cDM_L3N256_DM_power00_sigma1_dir_6', '/home/ryan/Data/run_2

In [4]:

def parsePower(powerType):
    stripped = powerType[5:]
    negFlag = False
    powers = [] 
    for char in stripped:
        if char == 'm':
            negFlag = True
            continue
        if negFlag:
            powers.append(-int(char))
            negFlag = False
        else:
            powers.append(int(char))

    return powers

def parseSigma(sigmaType):
    return float(sigmaType[5:])


In [5]:
def get_mass_profile(f, nbins=1000):

    Subhalo=f.get('Subhalo')
    SubhaloMass=np.array(Subhalo['SubhaloMass'])
    SubhaloMass=10**10*SubhaloMass #converting to solar masses

    Mass_Bins = np.linspace(0, np.amax(SubhaloMass), num=nbins)
    N_halo = np.array([ (SubhaloMass > mbin).sum() for mbin in Mass_Bins ])
    return Mass_Bins, N_halo

def get_mvel_profile(f, nbins=1000):

    Subhalo=f.get('Subhalo') #Subhalo is the group, SubhaloVMas is the dataset inside the group
    SubhaloVMax=np.array(Subhalo['SubhaloVmax']) #km/s

    #Take Absolute Values of the velocities
    MaxVel=np.absolute(SubhaloVMax)

    MaxVel_Bin = np.linspace(0, np.amax(MaxVel), num=nbins)
    N_halo = np.array([ (SubhaloVMax > vbin).sum() for vbin in MaxVel_Bin])
    return MaxVel_Bin, N_halo

In [6]:
for subdir in tqdm(subdir_list):
    
    #run name is just folder name
    run_name = subdir.split('/')[-1]

    #reject folders that aren't runs
    if 'run' not in run_name:
        continue
    
    fpath = os.path.join(outdir, run_name)
    try:
        os.mkdir(fpath)
    except:
        print(f'{fpath} already exists!')
    print("Working on ", run_name)

    # read info from snapshots/fof 
    snaps = sorted(glob.glob(subdir + '/snap*'))
    fofs = sorted(glob.glob(subdir + '/fof*'))
    
    #get info from filename
    splitted = run_name.split('_')

    dm_type = splitted[1]

    # boxPart = splitted[2] # read from snapshot

    baryon_type = splitted[3]


    run_dict = {
        "run name": run_name,
        "DM type": dm_type,
        "baryon_type": baryon_type
    }

    if dm_type == 'CDM':
        otherInfo = splitted[4:]
        run_dict['otherInfo'] = otherInfo
    elif (dm_type == '2cDM') or (dm_type == 'SIDM'):
        powerType = splitted[4]
        powerLaws = parsePower(powerType)
        sigmaType = splitted[5]
        sigma = parseSigma(sigmaType)
        otherInfo = splitted[6:]

        temp = {
            "powerLaws": powerLaws,
            "sigma0": sigma,
            "otherInfo": otherInfo
        }

        run_dict.update(temp)

    with h5py.File(snaps[0], 'r') as f:
        boxSize = int((f['Header'].attrs['BoxSize'])) # in kpc
        NPart = int(np.cbrt(f['Header'].attrs['NumPart_Total'][1]))
        mass_resolution = f['Header'].attrs['MassTable'][1] * 10**10 # in solar masses
        softening_length = boxSize / NPart / 29
        temp = {
            "BoxSize": boxSize,
            "NPart": NPart,
            "Mass Resolution": mass_resolution,
            "Softening Length": softening_length
        }

        run_dict.update(temp)


    redshifts = []
    for i, fof in enumerate(fofs):
        with h5py.File(fof, 'r') as f:
            
            redshifts.append( float("{:.2f}".format(f['Header'].attrs['Redshift'])) )

            try:
                mbins, mcount = get_mass_profile(f)
                tname = os.path.join(fpath, "mass_profile_{}.txt".format(i))
                np.savetxt(tname, (mbins, mcount))
                vbins, vcount = get_mvel_profile(f)
                tname = os.path.join(fpath, "vel_profile_{}.txt".format(i))
                np.savetxt(tname, (vbins, vcount))
            except KeyError:
                continue

    run_dict['redshifts'] = redshifts

    fname = os.path.join(fpath, "run_info.json")
    with open(fname, 'w') as outfile:
        json.dump(run_dict, outfile)

  0%|          | 0/73 [00:00<?, ?it/s]

../../data_prods/run_2cDM_L3N256_DM_powerm2m2_sigma0.1 already exists!
Working on  run_2cDM_L3N256_DM_powerm2m2_sigma0.1


  1%|▏         | 1/73 [00:00<00:49,  1.44it/s]

../../data_prods/run_2cDM_L3N512_HY_power00_sigma1 already exists!
Working on  run_2cDM_L3N512_HY_power00_sigma1


  3%|▎         | 2/73 [00:02<01:19,  1.11s/it]

...

 97%|█████████▋| 71/73 [01:11<00:03,  1.84s/it]

Working on  run_CDM_L3N256_DM_dir_3


 99%|█████████▊| 72/73 [01:12<00:01,  1.52s/it]

Working on  run_2cDM_L3N256_DM_power00_sigma1_dir_9


100%|██████████| 73/73 [01:13<00:00,  1.01s/it]
